In [7]:
import pandas as pd  
import numpy as np  
from sklearn.ensemble import RandomForestClassifier  
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score  
import matplotlib.pyplot as plt  
import seaborn as sns  
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import StratifiedKFold
import tqdm
import os
import itertools
from sklearn.metrics import r2_score
import random

In [27]:
# Load the climate and landscape data  
file_path = 'U:\Hydrological Data\EStreams\Data 071025\climate_landscape_071025.csv'

# Read the file
climate_df = pd.read_csv(file_path)
climate_df.set_index("basin_id", inplace=True)
climate_df

,pet_mean,aridity,p_seasonality,T_range,SI_PET,phi_P_PET,annual_intensity_PM_median,annual_median_fraction_snow,interannual_var_fraction_snow,longest_duration_annual_snow_cover_median,...,steep_area_fra,elon_ratio,strm_dens,reservoir_storage,PM_mean,lakes_area_relative_to_catchA,First_wd_sin,First_wd_cos,Last_wd_sin,Last_wd_cos
basin_id,,,,,,,,,,,,,,,,,,,,,
FR003106,2.345796,1.104261,-0.479096,34.88,0.522981,4,0.000000,0.000000,0.000000,0.0,...,75.447,0.596,0.161,0.0,0.000000,0.000000,0.051584,0.998669,-0.132924,0.991126
PL000980,1.904948,1.175267,0.387163,51.90,0.753597,0,2.901667,0.059087,0.000902,36.0,...,0.000,0.605,0.202,0.0,0.096485,13.979592,0.221772,0.975099,-0.308812,0.951123
DENW1113,1.931105,0.784515,-0.019418,45.03,0.678843,5,3.333333,0.028516,0.000940,16.0,...,0.062,0.545,0.000,0.0,0.088173,0.000000,0.321058,0.947060,-0.275913,0.961183
DK000210,1.688305,0.872153,0.110275,42.13,0.736734,3,2.782143,0.026101,0.000849,13.0,...,0.033,0.684,0.164,0.0,0.060994,24.519104,0.312900,0.949786,-0.192322,0.981332
FR002552,2.670915,1.153719,-0.113840,41.21,0.562106,3,0.336147,0.025214,0.000429,243.0,...,59.444,0.525,0.149,0.0,0.063449,0.000000,0.017202,0.999852,-0.004301,0.999991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ISGR0020,0.767475,0.252966,-0.238482,34.41,0.822038,3,2.032680,0.401170,0.004641,243.0,...,3.430,0.439,0.230,0.0,1.044451,3.313979,0.998377,0.056952,-0.996210,0.086977
ISGR0021,0.824686,0.314798,-0.281849,33.76,0.830120,5,1.938378,0.343318,0.004635,191.0,...,11.571,0.471,0.167,0.0,0.745907,0.000000,0.999514,0.031174,-0.971395,-0.237468
ISGR0022,0.707926,0.214820,-0.266101,35.37,0.824619,5,2.403437,0.480625,0.004264,243.0,...,2.910,0.502,0.217,0.0,1.275957,2.706113,0.997250,0.074117,-0.986501,0.163757


In [28]:
# Load the cluster labels data (adjust the filename if needed)    
file_pathh = 'U:\Hydrological Data\EStreams\Data 071025\hydro_sign_clustered_081025.csv'

# Read the file
clusters_df = pd.read_csv(file_pathh)
clusters_df.set_index("basin_id", inplace=True)
clusters_df

,q_corr,q_mean,q_runoff_ratio,q_elas_Sawicz,q_elas_Sankarasubramanian,slope_sawicz,slope_yadav,slope_mcmillan,slope_addor,baseflow_index,...,C_k10_AllS_NOClimate,C_k10_AllS_NOClimate_remapped,C_k9_AllS_NOClimate,C_k9_AllS_NOClimate_remapped,C_k8_AllS_NOClimate,C_k8_AllS_NOClimate_remapped,C_k7_AllS_NOClimate,C_k7_AllS_NOClimate_remapped,C_k6_AllS_NOClimate,C_k6_AllS_NOClimate_remapped
basin_id,,,,,,,,,,,,,,,,,,,,,
FR003106,0.322,1.153,0.485,0.618,0.900,4.154,2.106,4.154,4.271,0.481,...,2,4,0,3,6,7,1,4,1,4
PL000980,-0.034,0.540,0.333,0.159,0.532,2.362,1.619,2.362,2.382,0.676,...,4,0,7,0,5,2,3,0,3,1
DENW1113,0.254,0.857,0.346,0.865,1.539,2.129,1.349,2.129,2.145,0.527,...,0,1,0,3,0,0,1,4,1,4
DK000210,0.023,0.644,0.334,1.702,1.275,2.524,1.893,2.524,2.500,0.701,...,7,7,1,7,7,4,2,5,2,3
FR002552,0.229,0.597,0.269,2.790,2.420,3.757,1.695,3.757,3.673,0.514,...,2,4,0,3,1,3,1,4,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ISGR0020,0.012,2.855,0.928,-0.105,0.291,0.333,0.328,0.333,0.356,0.964,...,3,6,3,5,3,5,4,2,4,2
ISGR0021,0.026,2.273,0.866,0.550,0.580,0.969,0.774,0.969,0.958,0.742,...,3,6,3,5,3,5,4,2,4,2
ISGR0022,-0.108,2.229,0.663,0.341,0.519,1.421,1.135,1.421,1.415,0.760,...,3,6,3,5,3,5,4,2,4,2


In [33]:
X = climate_df 
y = clusters_df['C_k10_AllS_NOClimate_remapped']  

## 10 clusters

In [34]:
# Storage for results
results_10 = []
all_results_10 = []

In [ ]:
# For 10 clusters
X = climate_df 
y = clusters_df['C_k10_AllS_NOClimate_remapped']  

#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define parameter grid
param_ranges = {
    'n_estimators': (25, 500),         # inclusive bounds
    'max_depth': (5, 60),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 15)
}

# Iterations 
n_iterations = 5000
for i in range(n_iterations):
    # Randomly sample parameters
   # Sampling random integers from the range in each iteration
    params = {
        'n_estimators': random.randint(*param_ranges['n_estimators']),
        'max_depth': random.randint(*param_ranges['max_depth']),
        'min_samples_split': random.randint(*param_ranges['min_samples_split']),
        'min_samples_leaf': random.randint(*param_ranges['min_samples_leaf'])
    }
    print(f"[{i+1}] Working with parameter set: n_estimators={params['n_estimators']:.2f},max_depth={params['max_depth']}, min_samples_split={params['min_samples_split']:.2f},min_samples_leaf={params['min_samples_leaf']:.2f}")
    # Train model
    model = RandomForestClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        random_state=42,
        class_weight='balanced', 
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Compute training accuracy
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    #Store results
    all_results_10.append({
        'train_acc': train_acc,
        'test_acc': test_acc,
        'params': params
    })



In [36]:
# save 
results_df_10 = pd.DataFrame(all_results_10)
results_df_10.to_csv(r'C:\Users\juliarudlang\OneDrive - Delft University of Technology\Documents\temp_rf_run_10C_5000.csv')

## 9 clusters

In [ ]:
# Empty list to store results
all_results_9 = []

In [ ]:
# For 9 clusters

X = climate_df 
y = clusters_df['C_k9_AllS_NOClimate_remapped']  
#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Define parameter grid
param_ranges = {
    'n_estimators': (25, 500),         # inclusive bounds
    'max_depth': (5, 60),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 15)
}
# Iterations 
n_iterations = 5000
for i in range(n_iterations):
    # Randomly sample parameters
   # Sampling random integers from the range in each iteration
    params = {
        'n_estimators': random.randint(*param_ranges['n_estimators']),
        'max_depth': random.randint(*param_ranges['max_depth']),
        'min_samples_split': random.randint(*param_ranges['min_samples_split']),
        'min_samples_leaf': random.randint(*param_ranges['min_samples_leaf'])
    }
    print(f"[{i+1}] Working with parameter set: n_estimators={params['n_estimators']:.2f},max_depth={params['max_depth']}, min_samples_split={params['min_samples_split']:.2f},min_samples_leaf={params['min_samples_leaf']:.2f}")
    # Train model
    model = RandomForestClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        random_state=42,
        class_weight='balanced', 
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Compute training accuracy
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    #Store results
    all_results_9.append({
        'train_acc': train_acc,
        'test_acc': test_acc,
        'params': params
    })



In [ ]:
# Save results
results_df_9 = pd.DataFrame(all_results_9)
results_df_9.to_csv(r'C:\Users\juliarudlang\OneDrive - Delft University of Technology\Documents\temp_rf_run_9C_5000.csv')

## 8 clusters

In [ ]:
# Emptu list to stre results
all_results_8 = []

In [ ]:
# For 8 clusters

X = climate_df 
y = clusters_df['C_k8_AllS_NOClimate_remapped']  
#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Define parameter grid
param_ranges = {
    'n_estimators': (25, 500),         # inclusive bounds
    'max_depth': (5, 60),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 15)
}
# Iterations 
n_iterations = 5000
for i in range(n_iterations):
    # Randomly sample parameters
   # Sampling random integers from the range in each iteration
    params = {
        'n_estimators': random.randint(*param_ranges['n_estimators']),
        'max_depth': random.randint(*param_ranges['max_depth']),
        'min_samples_split': random.randint(*param_ranges['min_samples_split']),
        'min_samples_leaf': random.randint(*param_ranges['min_samples_leaf'])
    }
    print(f"[{i+1}] Working with parameter set: n_estimators={params['n_estimators']:.2f},max_depth={params['max_depth']}, min_samples_split={params['min_samples_split']:.2f},min_samples_leaf={params['min_samples_leaf']:.2f}")
    # Train model
    model = RandomForestClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        random_state=42,
        class_weight='balanced', 
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Compute training accuracy
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    #Store results
    all_results_8.append({
        'train_acc': train_acc,
        'test_acc': test_acc,
        'params': params
    })



In [ ]:
# Save results
results_df_8 = pd.DataFrame(all_results_8)
results_df_8.to_csv(r'C:\Users\juliarudlang\OneDrive - Delft University of Technology\Documents\temp_rf_run_8C_5000.csv')

## 7 clusters

In [ ]:
# Empty list for storage of results
all_results_7 = []

In [ ]:
# For 7 clusters

X = climate_df 
y = clusters_df['C_k7_AllS_NOClimate_remapped']  
#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Define parameter grid
param_ranges = {
    'n_estimators': (25, 500),         # inclusive bounds
    'max_depth': (5, 60),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 15)
}
# Iterations 
n_iterations = 5000
for i in range(n_iterations):
    # Randomly sample parameters
   # Sampling random integers from the range in each iteration
    params = {
        'n_estimators': random.randint(*param_ranges['n_estimators']),
        'max_depth': random.randint(*param_ranges['max_depth']),
        'min_samples_split': random.randint(*param_ranges['min_samples_split']),
        'min_samples_leaf': random.randint(*param_ranges['min_samples_leaf'])
    }
    print(f"[{i+1}] Working with parameter set: n_estimators={params['n_estimators']:.2f},max_depth={params['max_depth']}, min_samples_split={params['min_samples_split']:.2f},min_samples_leaf={params['min_samples_leaf']:.2f}")
    # Train model
    model = RandomForestClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        random_state=42,
        class_weight='balanced', 
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Compute training accuracy
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    #Store results
    all_results_7.append({
        'train_acc': train_acc,
        'test_acc': test_acc,
        'params': params
    })



In [ ]:
# Save results
results_df_7 = pd.DataFrame(all_results_7)
results_df_7.to_csv(r'C:\Users\juliarudlang\OneDrive - Delft University of Technology\Documents\temp_rf_run_7C_5000.csv')

## 6 clusters

In [ ]:
# Empty storage for results
all_results_6 = []

In [ ]:
# For 6 clusters

X = climate_df 
y = clusters_df['C_k6_AllS_NOClimate_remapped']  
#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Define parameter grid
param_ranges = {
    'n_estimators': (25, 500),         # inclusive bounds
    'max_depth': (5, 60),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 15)
}
# Iterations 
n_iterations = 5000
for i in range(n_iterations):
    # Randomly sample parameters
   # Sampling random integers from the range in each iteration
    params = {
        'n_estimators': random.randint(*param_ranges['n_estimators']),
        'max_depth': random.randint(*param_ranges['max_depth']),
        'min_samples_split': random.randint(*param_ranges['min_samples_split']),
        'min_samples_leaf': random.randint(*param_ranges['min_samples_leaf'])
    }
    print(f"[{i+1}] Working with parameter set: n_estimators={params['n_estimators']:.2f},max_depth={params['max_depth']}, min_samples_split={params['min_samples_split']:.2f},min_samples_leaf={params['min_samples_leaf']:.2f}")
    # Train model
    model = RandomForestClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        random_state=42,
        class_weight='balanced', 
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Compute training accuracy
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    #Store results
    all_results_6.append({
        'train_acc': train_acc,
        'test_acc': test_acc,
        'params': params
    })



In [ ]:
# Save results
results_df_6 = pd.DataFrame(all_results_6)
results_df_6.to_csv(r'C:\Users\juliarudlang\OneDrive - Delft University of Technology\Documents\temp_rf_run_6C_5000.csv')

## End